# client

> SDK to interact with the Agora-Fewsats Marketplace API

In [ ]:
#| default_exp client

## Class

In [ ]:
#| export


import httpx
from typing import Dict, List
from agora.config import *
from agora.crypto import *

from fastcore.utils import *


In [ ]:
# Test values

test_product_id = "678f61ec0205c9203bc07a45"
test_variant_id = 42884194664644

test_shipping_address = {
                "addressFirst": "123 Main St",
                "city": "New York",
                "state": "NY",
                "country": "US",
                "addressName": "Home",
                "zipCode": "10001"
            }
test_user = {
                "firstname": "John",
                "lastname": "Doe",
                "email": "john@example.com",
                "_id": "user123"
            }

In [ ]:
#| export


class Agora:
    """
    Client for the Agora-Fewsats Marketplace API.
    
    This client provides methods to interact with the Agora-Fewsats Marketplace API,
    including product search, cart management, and checkout functionality.
    """
    
    def __init__(self, base_url: str = "https://agora-backend.replit.app/api/v1", private_key: str = ''):
        """
        Initialize the Agora Marketplace client.
        
        Args:
            base_url: The base URL of the Agora-Fewsats Backend API.
            private_key: The private key of the Agora-Fewsats Backend API. If not provided,
                         a random UUID will be generated.
        """
        self.base_url = base_url

        cfg = get_cfg()

        if private_key: self.pk, self.pub = from_pk_hex(private_key) # if provided use the private key
        elif cfg.priv: self.pk, self.pub = from_pk_hex(cfg.priv) # if not provided use the private key from the config file
        else: 
            self.pk, self.pub = generate_keys()
            save_cfg({'priv': priv_key_hex(self.pk)})

        self.customer_user_id = self.pub[:24] # agora uses mongoDB so the related user-objectId needs to be 24 characters

        self.client = httpx.Client(timeout=30.0)
        self.client.headers.update({"customuserid": self.customer_user_id})

    def _make_request(self, method: str, endpoint: str, **kwargs) -> httpx.Response:
        """
        Make a request to the Agora-Fewsats Backend API.
        
        Args:
            method: The HTTP method to use.
            endpoint: The API endpoint to call.
            **kwargs: Additional arguments to pass to httpx.
            
        Returns:
            The response from the API.
        """
        url = f"{self.base_url}/{endpoint}"
        response = self.client.request(method, url, **kwargs)
        
        
        return response

In [ ]:
a = Agora()
# a = Agora(base_url="http://localhost:8000/api/v1")

## Methods

### Search product

In [ ]:
#| export

@patch
def search_products(self: Agora, query: str, count: int = 20, page: int = 1, 
                        price_min: int = 0, price_max: int = None, 
                        sort: str = None, order: str = None) -> Dict:
    """
    Search for products.
    
    Args:
        query: The search query.
        count: The number of products to return per page.
        page: The page number.
        price_min: The minimum price.
        price_max: The maximum price.
        sort: The sort field.
        order: The sort order.
        
    Returns:
        The search results.
    """
    params = {
        "q": query,
        "count": count,
        "page": page,
        "price_min": price_min
    }
    
    if price_max is not None:
        params["price_max"] = price_max
    if sort is not None:
        params["sort"] = sort
    if order is not None:
        params["order"] = order
    
    print(params)
    return self._make_request("GET", "search", params=params)


In [ ]:
r = a.search_products("shirt", price_min=1, price_max=10)
r.status_code, r.json()

{'q': 'shirt', 'count': 20, 'page': 1, 'price_min': 1, 'price_max': 10}


(200,
 {'status': 'success',
  'Products': [{'_id': '667b4b828db86e6495d1aa9d',
    'name': 'Ann Arbor, Michigan Shirt (Discontinued)',
    'storeName': 'Jupmode',
    'brand': 'Jupmode',
    'slug': 'ann-arbor-michigan-shirt-8cdf02a1-2e00-4de0-9778-936ff4427329-1719356290515',
    'price': 8,
    'source': 'shopify',
    'images': ['https://cdn.shopify.com/s/files/1/1010/8058/products/Ann-Arbor.jpg?v=1582504406'],
    'url': 'https://jupmode.com/products/ann-arbor-michigan-shirt',
    'agoraScore': 100,
    'priceHistory': [{'price': 8,
      'date': '2024-10-19T10:32:27.912Z',
      '_id': '67138abb6ca0a1804a27c3ab'},
     {'price': 24,
      'date': '2024-06-25T23:05:43.793Z',
      '_id': '67138abb6ca0a1804a27c3aa'}],
    '_rankingScore': 0.693359375,
    '_combinedScoreData': {'clipScore': 0.646484375,
     'cohereScore': 0.693359375,
     'maxScore': 0.693359375,
     'bestEmbedding': 'cohere',
     'position': 0},
    '_adjustedScore': 0.693359375},
   {'_id': '67c86bb5c31ea3271

In [ ]:
p = r.json()["Products"][0]
p

{'_id': '667b4b828db86e6495d1aa9d',
 'name': 'Ann Arbor, Michigan Shirt (Discontinued)',
 'storeName': 'Jupmode',
 'brand': 'Jupmode',
 'slug': 'ann-arbor-michigan-shirt-8cdf02a1-2e00-4de0-9778-936ff4427329-1719356290515',
 'price': 8,
 'source': 'shopify',
 'images': ['https://cdn.shopify.com/s/files/1/1010/8058/products/Ann-Arbor.jpg?v=1582504406'],
 'url': 'https://jupmode.com/products/ann-arbor-michigan-shirt',
 'agoraScore': 100,
 'priceHistory': [{'price': 8,
   'date': '2024-10-19T10:32:27.912Z',
   '_id': '67138abb6ca0a1804a27c3ab'},
  {'price': 24,
   'date': '2024-06-25T23:05:43.793Z',
   '_id': '67138abb6ca0a1804a27c3aa'}],
 '_rankingScore': 0.693359375,
 '_combinedScoreData': {'clipScore': 0.646484375,
  'cohereScore': 0.693359375,
  'maxScore': 0.693359375,
  'bestEmbedding': 'cohere',
  'position': 0},
 '_adjustedScore': 0.693359375}

### Get product detail

In [ ]:
#| export

@patch    
def get_product_detail(self: Agora, slug: str) -> Dict:
    """
    Get details for a specific product.
    
    Args:
        slug: The product slug.
        
    Returns:
        The product details.
    """
    params = {"slug": slug}
    return self._make_request("GET", "product-detail", params=params)


In [ ]:
r = a.get_product_detail(p['slug'])
r.json()

{'status': 'success',
 'product': {'_id': '667b4b828db86e6495d1aa9d',
  'keywords': [],
  'name': 'Ann Arbor, Michigan Shirt (Discontinued)',
  'storeName': 'Jupmode',
  'brand': 'Jupmode',
  'tags': ['ann arbor',
   'faire',
   'go blue',
   'michigan',
   'spring savings',
   'springsavings',
   'vintage',
   'wolverines'],
  'categories': ['Sports & Outdoors', 'Fan Shop'],
  'description': "<p>We like to think of Ann Arbor as Toledo's hip BFF up north. Whether you're heading to\xa0a game at the Big House, checking out one of their art fairs, treating yourself to some food at Zingerman's, or enjoying that outdoor lifestyle at one of\xa0their many parks,\xa0Ann Arbor is a great place to be.\xa0</p>",
  'country': 'US',
  'currency': 'USD',
  'slug': 'ann-arbor-michigan-shirt-8cdf02a1-2e00-4de0-9778-936ff4427329-1719356290515',
  'price': '8.00',
  'externalProductId': '6233296069',
  'published_at': '2016-09-16T17:15:00.000Z',
  'last_updated': '2024-06-25T22:58:10.000Z',
  'verifiedA

### Get cart

In [ ]:
#| export

@patch
def get_cart(self: Agora) -> Dict:
    """
    Get the current user's cart.
    
    Returns:
        The cart details.
    """
    return self._make_request("GET", "cart")


In [ ]:
r = a.get_cart()
r.json()

{'created_at': 1743050017, 'updated_at': 1743050017, 'items': []}

### Add to cart

In [ ]:
#| export
  
@patch
def add_to_cart(self: Agora, slug: str, product_id: str, variant_id: str = None, quantity: int = 1) -> Dict:
    """
    Add an item to the user's cart. Somem products do not have variants, in such cases use the product_id as variant_id too.
    
    Args:
        slug: The product slug.
        product_id: The product ID.
        variant_id: The product variant ID.
        quantity: The quantity to add.
        
    Returns:
        The updated cart.
    """
    item = {
        "slug": slug,
        "product_id": product_id,
        "variant_id": variant_id if variant_id else product_id,
        "quantity": quantity,
    }
    
    return self._make_request("POST", "cart/items", json=item)


In [ ]:
r = a.add_to_cart(p['slug'], p['_id'])
r.status_code, r.json()

(200,
 {'created_at': 1743050017,
  'updated_at': 1743050017,
  'items': [{'product_id': '667b4b828db86e6495d1aa9d',
    'variant_id': '667b4b828db86e6495d1aa9d',
    'quantity': 1,
    'title': 'Ann Arbor, Michigan Shirt (Discontinued) by Jupmode',
    'description': 'Ann Arbor, Michigan Shirt (Discontinued) by Jupmode from Jupmode',
    'amount': 800,
    'currency': 'USD',
    'slug': 'ann-arbor-michigan-shirt-8cdf02a1-2e00-4de0-9778-936ff4427329-1719356290515'}]})

### Update cart item

In [ ]:
#| export

@patch
def update_cart_item(self: Agora, slug: str, product_id: str, variant_id: str, quantity: int) -> Dict:
    """
    Update the quantity of an item in the cart. Some products do not have variants, in such cases use the product_id as variant_id too.
    
    Args:
        slug: The product slug.
        product_id: The product ID.
        variant_id: The product variant ID.
        quantity: The new quantity.
        
    Returns:
        The updated cart.
    """
    params = {
        "slug": slug,
        "product_id": product_id,
        "variant_id": variant_id,
        "quantity": quantity
    }
    
    return self._make_request("PUT", "cart/items", params=params)
    


In [ ]:
r = a.update_cart_item(p['slug'], p['_id'], p['_id'], 1)
r.status_code, r.json()


(200,
 {'created_at': 1743050017,
  'updated_at': 1743050018,
  'items': [{'product_id': '667b4b828db86e6495d1aa9d',
    'variant_id': '667b4b828db86e6495d1aa9d',
    'quantity': 1,
    'title': 'Ann Arbor, Michigan Shirt (Discontinued) by Jupmode',
    'description': 'Ann Arbor, Michigan Shirt (Discontinued) by Jupmode from Jupmode',
    'amount': 800,
    'currency': 'USD',
    'slug': 'ann-arbor-michigan-shirt-8cdf02a1-2e00-4de0-9778-936ff4427329-1719356290515'}]})

In [ ]:
#| export

@patch
def clear_cart(self: Agora) -> Dict:
    """
    Clear all items from the cart.
    
    Returns:
        The response from the API.
    """
    return self._make_request("DELETE", "cart")

In [ ]:
r = a.clear_cart()
r.status_code, r.json()

(200, {'status': 'success', 'data': None, 'message': 'Cart cleared'})

### Buy now

In [ ]:
#| export

@patch
def buy_now(self: Agora, slug: str, product_id: str, variant_id: str, shipping_address: Dict,
            user: Dict, quantity: int = 1) -> Dict:
    """
    Purchase a product directly. Some products do not have variants, in such cases use the product_id as variant_id too.
    
    Args:
        slug: The product slug.
        product_id: The product ID.
        variant_id: The product variant ID.
        quantity: The quantity to purchase.
        shipping_address: The shipping address.
        user: The user information.
        
    Example:
        shipping_address = {
            "addressFirst": "123 Main St",
            "city": "New York",
            "state": "NY",
            "country": "US",
            "addressName": "Home",
            "zipCode": "10001"
        }
        
        user = {
            "firstname": "John",
            "lastname": "Doe",
            "email": "john@example.com",
        }
        
        a.buy_now(product_id, variant_id, shipping_address, user)
        
    Returns:
        The payment information.
    """

        
    request_data = {
        "slug": slug,
        "product_id": product_id,
        "variant_id": variant_id,
        "quantity": quantity,
        "shipping_address": shipping_address,
        "user": user
    }
    
    return self._make_request("POST", "buy-now", json=request_data)

In [ ]:
r = a.buy_now(p['slug'], p['_id'], p['_id'], test_shipping_address, test_user, quantity=1)
offers = r.json()
r.status_code, r.json()

(402,
 {'offers': [{'id': 'c4cbe0e9-6f57-459a-a5d2-7bae4d5373d4',
    'amount': 800,
    'currency': 'USD',
    'description': 'Ann Arbor, Michigan Shirt (Discontinued) by Jupmode from Jupmode',
    'title': 'Ann Arbor, Michigan Shirt (Discontinued) by Jupmode',
    'payment_methods': ['lightning', 'credit_card'],
    'type': 'one-off'}],
  'payment_context_token': '863656f7-f961-4467-8c5c-25b68b11afbf',
  'payment_request_url': 'https://api.fewsats.com/v0/l402/payment-request',
  'version': '0.2.2'})

In [ ]:
# shipping_address = {
#     "addressFirst": "Anselm Clave 1",
#     "city": "Tordera", 
#     "state": "Barcelona",
#     "country": "ES",
#     "addressName": "Home",
#     "zipCode": "08490"
# }

# user = {
#     "firstname": "Pol",
#     "lastname": "Alvarez Vecino",
#     "email": "pol.avms@gmail.com",
#     "_id": "user123"
# }

# # Call buy_now with the parameters in the correct order
# r = a.buy_now(
#     slug="disappearing-ink-1-count-c2e1b357-1e86-46c7-bee0-328aefc5fe3f-1742527478059",
#     product_id=7660653936801,
#     variant_id=7660653936801,
#     shipping_address=shipping_address,
#     user=user,
#     quantity=1  # Using default quantity of 1 since not specified
# )
# r.status_code, r.json()

### Webhook test

In [ ]:
# test webhook 
offers = r.json()

from datetime import datetime, timezone

# Your existing data from buy_now
offer_id = offers['offers'][0]['id']
payment_context_token = offers['payment_context_token']
amount = offers['offers'][0]['amount']
currency = offers['offers'][0]['currency']

# Create a webhook payload
payload = {
    "offer_id": offer_id,
    "payment_context_token": payment_context_token,
    "amount": amount,
    "currency": currency,
    "status": "succeeded",  # Use 'succeeded' to simulate successful payment
    "timestamp": datetime.now(timezone.utc).isoformat()
}

# Send the request to your webhook endpoint
url = f"{a.base_url}/webhook/payment"
response = httpx.post(url, json=payload, timeout=120.0)


### Get user orders

In [ ]:
#| export

@patch
def get_user_orders(self: Agora) -> List[Dict]:
    """
    Get all orders for the current user.
    
    Returns:
        A list of orders.
    """
    return self._make_request("GET", f"users/orders")

In [ ]:
r = a.get_user_orders()
orders = r.json()
r.status_code, orders

(200,
 [{'payment_context_token': '863656f7-f961-4467-8c5c-25b68b11afbf',
   'status': 'paid',
   'created_at': 1743050019,
   'updated_at': 1743050019,
   'external_id': '67e4d523aae5be2cc9ba85fb',
   'items': [{'product_id': '667b4b828db86e6495d1aa9d',
     'variant_id': '667b4b828db86e6495d1aa9d',
     'quantity': 1,
     'title': 'Ann Arbor, Michigan Shirt (Discontinued) by Jupmode',
     'description': 'Ann Arbor, Michigan Shirt (Discontinued) by Jupmode from Jupmode',
     'amount': 800,
     'currency': 'USD',
     'slug': 'ann-arbor-michigan-shirt-8cdf02a1-2e00-4de0-9778-936ff4427329-1719356290515'}]}])

### Get order

In [ ]:
#| export
   
@patch
def get_order(self: Agora, external_id: str) -> Dict:
    """
    Get details for a specific order.
    
    Args:
        external_id: The external ID of the order.
        
    Returns:
        The order details.
    """
    return self._make_request("GET", f"orders/{external_id}")

In [ ]:
r = a.get_order(orders[0]['external_id'])
r.status_code, r.json()

(200,
 {'payment_context_token': '863656f7-f961-4467-8c5c-25b68b11afbf',
  'status': 'paid',
  'created_at': 1743050019,
  'updated_at': 1743050019,
  'external_id': '67e4d523aae5be2cc9ba85fb',
  'items': [{'product_id': '667b4b828db86e6495d1aa9d',
    'variant_id': '667b4b828db86e6495d1aa9d',
    'quantity': 1,
    'title': 'Ann Arbor, Michigan Shirt (Discontinued) by Jupmode',
    'description': 'Ann Arbor, Michigan Shirt (Discontinued) by Jupmode from Jupmode',
    'amount': 800,
    'currency': 'USD',
    'slug': 'ann-arbor-michigan-shirt-8cdf02a1-2e00-4de0-9778-936ff4427329-1719356290515'}]})

### Get user & address info

In [ ]:
#| export

@patch
def get_user_info(self: Agora) -> Dict:
    """
    Get the current user's profile and shipping addresses.
    
    Returns:
        Dict containing user profile info (firstname, lastname, email) and list of shipping addresses
    """
    return self._make_request("GET", "user/info")

In [ ]:
r = a.get_user_info()
r.status_code, r.json()

(200,
 {'firstname': 'John',
  'lastname': 'Doe',
  'email': 'john@example.com',
  'addresses': [{'id': 1,
    'addressName': 'Home',
    'addressFirst': '123 Main St',
    'city': 'New York',
    'state': 'NY',
    'country': 'US',
    'zipCode': '10001'}]})

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()